In [453]:
import pandas as pd

In [454]:
import numpy as np

In [473]:
homicidios = pd.read_csv("data/censo_2000_pe/agressoes_arma_de_fogo_2000.csv")
homicidios = homicidios.rename(index=str, columns={"2000": "ano_2000"})
homicidios = homicidios.replace('-', np.nan)
homicidios['ano_2000'] = homicidios.ano_2000.astype("float64")
#homicidios

In [474]:
populacao = pd.read_csv("data/estimativa_populacao/populacao_pe_municipios.csv")
#populacao

In [475]:
microrregioes = pd.read_csv("data/microrregioes/municipios_microrregioes_pernambuco.csv")
#microrregioes

In [476]:
aux2 = pd.read_csv("data/censo_2000_pe/csv/edu_alunos_por_serie.csv")
#aux2.head()
codigo = aux2[['municipio', 'codigo_unidade_geografica']]
codigo = codigo.copy()
codigo['codigo_unidade_geografica'] = codigo['codigo_unidade_geografica'].apply(lambda x: int(x/10))
#codigo

In [477]:
codigo['municipio'] = codigo['municipio'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
microrregioes['municipio'] = microrregioes['municipio'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
microrregioes['microrregiao'] = microrregioes['microrregiao'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
populacao['municipio'] = populacao['municipio'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')


In [478]:
tb_parcial = pd.concat([codigo.set_index("municipio"),\
                                microrregioes.set_index("municipio"), populacao.set_index("municipio")], axis= 1)
tb_parcial = tb_parcial.reset_index()
tb_parcial = tb_parcial.rename(index=str, columns={"index": "municipio"})

In [479]:
tb_parcial.head()

,municipio,codigo_unidade_geografica,microrregiao,pop
0,Abreu e Lima,260005,Recife,83486
1,Afogados da Ingazeira,260010,Vale do Pajeu,34524
2,Afranio,260020,Petrolina,14227
3,Agrestina,260030,Brejo,18713
4,Agua Preta,260040,Mata Meridional,25353


In [581]:
def filtro(codigo):
    aux=[]
    if (codigo >= 260001 and codigo <= 269999):
        aux.append(int(codigo))
    df_newcode = pd.DataFrame(aux, columns=["codigo_un"])
    return df_newcode

In [582]:
n_homicidios = pd.concat(map(lambda a: filtro(a.codigo_unidade_geografica).\
                             assign(homicidio = a.ano_2000), homicidios.itertuples()), ignore_index=True)
#n_homicidios.head()

In [583]:
tabela_ref = pd.concat([tb_parcial.set_index("codigo_unidade_geografica"),\
                                n_homicidios.set_index("codigo_un")], axis= 1)

In [584]:
tabela_ref = tabela_ref.reset_index()
tabela_ref = tabela_ref.rename(columns={"index": "codigo_un"})
tabela_ref.head()

,codigo_un,municipio,microrregiao,pop,homicidio
0,260005,Abreu e Lima,Recife,83486,32.0
1,260010,Afogados da Ingazeira,Vale do Pajeu,34524,8.0
2,260020,Afranio,Petrolina,14227,1.0
3,260030,Agrestina,Brejo,18713,8.0
4,260040,Agua Preta,Mata Meridional,25353,21.0


In [585]:
def taxa(populacao, homicidios):
    aux=[]
    taxa_h = 100000*homicidios/populacao
    aux.append(taxa_h)
    df_taxa = pd.DataFrame(aux, columns=["taxa_homicidio"])
    return df_taxa

In [586]:
taxa = pd.concat(map(lambda a: taxa(a.pop, a.homicidio),\
                       tabela_ref.itertuples()), ignore_index=True)

In [591]:
tabela = pd.concat([tabela_ref, taxa], axis=1)
tabela.head()

,codigo_un,municipio,microrregiao,pop,homicidio,taxa_homicidio
0,260005,Abreu e Lima,Recife,83486,32.0,38.329780
1,260010,Afogados da Ingazeira,Vale do Pajeu,34524,8.0,23.172286
2,260020,Afranio,Petrolina,14227,1.0,7.028889
3,260030,Agrestina,Brejo,18713,8.0,42.751029
4,260040,Agua Preta,Mata Meridional,25353,21.0,82.830434


In [593]:
tabela.to_csv('../2017-2-projeto-lbs4-eebls-ktcn/tabela_final.csv', index=False)